## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6722309243364431110
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6686052843
locality {
  bus_id: 1
  links {
  }
}
incarnation: 916830884635923346
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]






['/job:localhost/replica:0/task:0/device:GPU:0']

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [5]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\bangb\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [6]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(input_shape=(32, 32, 3), kernel_size=(3,3), filters=32, activation='relu', padding='same'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(kernel_size=(3,3), filters=32, activation='relu', padding='same'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

C:\Users\bangb\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
C:\Users\bangb\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
50000/50000 [==============================] - 9s 189us/step - loss: 1.3586 - acc: 0.5253
Epoch 2/100
50000/50000 [==============================] - 7s 140us/step - loss: 0.9456 - acc: 0.6700
Epoch 3/100
50000/50000 [==============================] - 7s 133us/step - loss: 0.7767 - acc: 0.7257
Epoch 4/100
50000/50000 [==============================] - 7s 134us/step - loss: 0.6444 - acc: 0.7741
Epoch 5/100
50000/50000 [==============================] - 7s 138us/step - loss: 0.5182 - acc: 0.8193
Epoch 6/100
50000/50000 [==============================] - 7s 138us/step - loss: 0.3989 - acc: 0.8618
Epoch 7/100
50000/50000 [==============================] - 8s 153us/step - loss: 0.3001 - acc: 0.8960
Epoch 8/100
50000/50000 [==============================] - 8s 153us/step - loss: 0.2179 - acc: 0.9237
Epoch 9/100
50000/50000 [==============================] - 7s 143us/step - loss: 0.1718 - 

50000/50000 [==============================] - 7s 143us/step - loss: 0.0172 - acc: 0.9949
Epoch 77/100
50000/50000 [==============================] - 7s 143us/step - loss: 0.0144 - acc: 0.9955
Epoch 78/100
50000/50000 [==============================] - 7s 143us/step - loss: 0.0132 - acc: 0.9962
Epoch 79/100
50000/50000 [==============================] - 7s 146us/step - loss: 0.0169 - acc: 0.9947
Epoch 80/100
50000/50000 [==============================] - 7s 144us/step - loss: 0.0230 - acc: 0.9937
Epoch 81/100
50000/50000 [==============================] - 7s 145us/step - loss: 0.0151 - acc: 0.9955
Epoch 82/100
50000/50000 [==============================] - 7s 142us/step - loss: 0.0175 - acc: 0.9947
Epoch 83/100
50000/50000 [==============================] - 7s 135us/step - loss: 0.0122 - acc: 0.9960
Epoch 84/100
50000/50000 [==============================] - 7s 137us/step - loss: 0.0102 - acc: 0.9969
Epoch 85/100
50000/50000 [==============================] - 7s 138us/step - loss: 0.01

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[6.5573812e-07, 8.7241843e-25, 5.7286506e-06, 9.9999309e-01,
        1.4608594e-12, 6.4676448e-27, 2.5277324e-16, 2.1288201e-27,
        5.0401479e-07, 2.9671297e-35]], dtype=float32)

In [8]:
classifier.evaluate(x_test, y_test)

10000/10000 [==============================] - 2s 160us/step


[3.458158515548706, 0.6645]